# Collision Avoidance - Live Demo(デモ)

このnotebookでは、ロボットが「フリー」か「ブロック」かを検出するトレーニング済みモデルを使用して、ロボットの衝突回避動作を有効にします。

## Load the trained model(学習済みモデルの読み込み)

ローカルの学習なので、すでに``best_model.pth``がローカルに存在している事を前提とします。

> Please make sure the file has uploaded fully before calling the next cell

Pytorch modelの初期化を下記コードでおこないます。

In [ ]:
import torch
import torchvision

model = torchvision.models.alexnet(pretrained=False)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

次に、``best_model.pth``から、学習した重みを読み込みます。

In [ ]:
model.load_state_dict(torch.load('best_model.pth'))

現在、モデルの重みは、CPUメモリーで実行されているので、下記コードでGPUに転送します。

In [ ]:
device = torch.device('cuda')
model = model.to(device)

### Create the preprocessing function(事前処理関数の作成)

モデルを読み込みましたが、まだわずかな問題があります。学習済みモデルのフォーマットと、カメラの形式と*完全に*一致しません。これを解消するために、
いくつかの*前処理*を行う必要があります。これらは、下記の手順になります。

1. BGRからRGBに変換
2. HWC layoutからCHW layoutに変換
3. トレーニング中に使ったのと同じパラメーターを使用して正規化します（カメラは[0、255]の範囲の値を提供し、ロードされた画像は[0、1]の範囲でトレーニングするため、255.0でスケーリングする必要があります
4. dataをCPUメモリからGPUメモリに転送します
5. バッチディメンションを追加する

In [ ]:
import cv2
import numpy as np

mean = 255.0 * np.array([0.485, 0.456, 0.406])
stdev = 255.0 * np.array([0.229, 0.224, 0.225])

normalize = torchvision.transforms.Normalize(mean, stdev)

def preprocess(camera_value):
    global device, normalize
    x = camera_value
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    x = x.transpose((2, 0, 1))
    x = torch.from_numpy(x).float()
    x = normalize(x)
    x = x.to(device)
    x = x[None, ...]
    return x

すばらしい、これでカメラ形式からneural networkのインプットのフォーマットに変換するための、pre-processing関数を定義する事ができました。

カメラを起動し、画面に表示します。カメラの起動には、大分なれてきましたね。また、ロボットがブロックされる確率が表示されるスライダーを作成します。

In [ ]:
!echo jetbot | sudo -S systemctl restart nvargus-daemon

In [ ]:
import traitlets
from IPython.display import display
import ipywidgets.widgets as widgets
from jetbot import Camera, bgr8_to_jpeg

camera = Camera.instance(width=224, height=224, fps=2)
image = widgets.Image(format='jpeg', width=224, height=224)
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

display(widgets.HBox([image, blocked_slider]))

モーターを制御するためにrobotインスタンスを生成します。

In [ ]:
from jetbot import Robot

robot = Robot()

次に、カメラの値が変更されるたびに呼び出される関数を作成します。この関数は下記手順で実行します。

1. カメラのイメージのPre-process
2. neural networkの実行
3. ニューラルネットワークの出力がブロックを示した場合は、左に曲がります。それ以外の場合は前進します。

In [ ]:
import torch.nn.functional as F
import time

def update(change):
    global blocked_slider, robot
    x = change['new'] 
    x = preprocess(x)
    y = model(x)
    
    # we apply the `softmax` function to normalize the output vector so it sums to 1 (which makes it a probability distribution)
    y = F.softmax(y, dim=1)
    
    prob_blocked = float(y.flatten()[0])
    
    blocked_slider.value = prob_blocked
    
    if prob_blocked < 0.5:
        robot.forward(0.4)
    else:
        robot.left(0.4)
    
    time.sleep(0.001)

すばらしい、neural networkの実行関数が生成できたので、あとは処理をカメラと関連づける必要がある。

それは `` observe``関数で実現します。

> 注意: このコードでロボットが動きだします。十分なスペースを確保してください。collision avoidanceが動くでしょう、nueral networkは、訓練されたデータと同じくらいに動くはずです。

``start jetbot``でカメラのupdateがおこなわれ、`` observe``関数で処理をカメラと関連づけます。

In [ ]:
import ipywidgets
import time

model_start_button = ipywidgets.Button(description='start jetbot')
model_stop_button = ipywidgets.Button(description='stop jetbot')

def start_model(c):
    update({'new': camera.value})  
    camera.observe(update, names='value')
model_start_button.on_click(start_model)
    
def stop_model(c):
    camera.unobserve(update, names='value')
    time.sleep(1)
    robot.stop()
    #robot.stop()
    #camera_link.unlink()
model_stop_button.on_click(stop_model)

model_widget = ipywidgets.VBox([
    ipywidgets.HBox([image, blocked_slider]),
    ipywidgets.HBox([model_start_button, model_stop_button])
])

display(model_widget)

驚くばかり！ロボットが接続されている場合、新しいカメラフレームごとに新しいコマンドが生成されるはずです。おそらく、ロボットを地面に置き、障害物に到達したときの動作を確認する事ができるでしょう。

もし、今の挙動を止めたい場合は、下記コードを実行し、このコールバックの関連付けをやめます。

In [ ]:
#import time

#camera.unobserve(update, names='value')

#time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

#robot.stop()

もしかしたら、videoをブラウザに転送しないでロボットを実行したいと思いますが、その場合は、下記のようにcameraをunlinkしてください。

In [ ]:
#camera_link.unlink()  # don't stream to browser (will still run camera)

streamingを続ける場合は、下記コードで続けます。

In [ ]:
#camera_link.link()  # stream to browser (wont run camera)

Robotを停止します。

In [ ]:
#robot.stop()

### Conclusion

以上がLive demoです。うまく行けば、ロボットが衝突をインテリジェントに回避できたことと思います。

avoiding collisionsが上手く行かない場合、失敗した箇所を見つけてください。美しさは、これら上手くいかないシナリオについてより多くのデータを収集すれば、ロボットはさらに良くなるはずです